# Using Quadratic Unconstrained Binary Optimasation (QUBO) to solve number partitioning
## Introduction
As has been shown in the presentation, the 'energy' of a system can be
_officially_ defined as:

\begin{equation}
    E = \frac{1}{2}Q \cdot (\boldsymbol{x} \otimes \boldsymbol{x}) +
     l \cdot \boldsymbol{x}.
\end{equation}

However, since we choose to depart from the scientifically accurate Ising model
toward non-physical problems, we can reduce this equation and simplify it to
prevent confusion. If we use the fact that the $x_i \in [0, 1]$, then we use
that ${x_i}^2 = x_i$, and we can use the _diagonal_ of the matrix $Q$ to save
our linear terms $l$. The equation becomes much shorter if we write it as:

\begin{equation}
    y = \sum_{i, j}^n Q_{ij} x_i x_j.
\end{equation}

Since the QUBO minimises $y$, it is now the goal to translate a problem into a
$Q$ matrix such that the desired result is found in the minimum of $y$. The 
annealer will handle all the increasing and decreasing in energy for us, 
so we can focus on only the $Q$.

This notebook will handle the case of number partitioning, which will be
explained in the following section.

## Number partitioning
### Description
Suppose we have a list of numbers $\mathcal{S} = \{s_1, s_2, \ldots, s_n\}$, how can we
divide the list into two subsets, such that the sum of each subset is
equal?

### Execution
Let $x_i = 0$ if $s_i$ is in group 0, and $1$ if otherwise. Let $A_0$ be the sum
of group 0, and $A_1$ the sum of group 1, we can define these as:

\begin{equation}
    A_1 = \sum_i^n x_i s_i, ~ A_0 = \sum_i^n s_i - x_i s_i.
\end{equation}

Since we want both sums to be equal, we want the difference
$\Delta = A_0 - A_1$ to be zero, so this would be a proper value to minimise.
However, the minimum of $\Delta$ is the largest negative number, representing 
the largest difference. To solve this, we will find the minimum of 
$\Delta^2$:

\begin{equation}
    \Delta^2 = \left( \sum_i^n s_i - 2 x_i s_i \right)^2
             = \left( \mathcal{A} - 2 \sum_i^n x_i s_i \right) ^2 ,
\end{equation}

where we have written $\sum_i^n s_i = \mathcal{A}$ as the full sum of
$\mathcal{S}$. This equation turns out to be writeable as:

\begin{equation}
    \Delta^2 = \mathcal{A}^2 + 4 Q \cdot (x \otimes x),
\end{equation}

where appendix A1 goes into the full derivation to this equation. We only 
need to know for now that $Q_{ii} = s_i (s_i - \mathcal{A})$ for the 
diagonal (or 'linear') terms, and $Q_{ij} = s_i s_j$ for the off-diagonal
(or 'quadratic') terms. We can now define $y = Q \cdot x \otimes x$ and 
use a QUBO-algorithm to find $\min y$.

### Code
We start by importing the important packages:

In [1]:
import numpy as np
import dimod

`numpy` will speak for itself. `dimod` is the package which will simulate
the annealer for us. We _can_ run our program on a DWave machine, using
`dwave-ocean-sdk`, but this requires a [full set-up][1] which we will not go
into at the moment.

#### Creating an Ising or QUBO model using `dimod`
In `dimod`, we can create a so-called `BinaryQuadraticModel` which, according
to the `dimod` [docs][2], "contains Ising and [QUBO] models used by samplers
such as the D-Wave system". We can call each of these models via:

[1]: https://docs.ocean.dwavesys.com/en/latest/getting_started.html
[2]: https://docs.ocean.dwavesys.com/projects/dimod/en/latest/reference/bqm/binary_quadratic_model.html

In [2]:
ising_model = dimod.BinaryQuadraticModel({}, {}, 0.0, dimod.SPIN)
qubo_model = dimod.BinaryQuadraticModel({}, {}, 0.0, dimod.BINARY)

The empty `dicts` and the `0.0` passed along in the constructor are the
_linear_ terms $Q_{ii}$, _quadratic_ terms $Q_{ij}$ and the _offset_,
respectively. We ignore the latter for this case.

As we have found in the introduction, our $Q$ looks as follows:
\begin{equation}
    Q_{ii} = s_i (s_i - \mathcal{A}); ~ Q_{ij} = s_i s_j,
\end{equation}
so we will translate this into one matrix:

In [3]:
# this method is not optimal, but structured for readability
def qubo_partition_matrix(numbers: np.ndarray) -> np.ndarray:
    """Takes a list of numbers which need to be partitioned and returns the 
    interaction matrix Q to pass to a QUBO."""
    matrix = np.zeros((len(numbers), len(numbers)))
    linear = numbers * (numbers - np.sum(numbers))  # becomes an n array
    quadratic = np.outer(numbers, numbers)  # becomes an nxn array

    matrix = quadratic
    np.fill_diagonal(matrix, linear)  # overwrite diagonal with linear terms

    return matrix

Now define the list of numbers which we want to partition, and create the
matrix $Q$, and create our `BinaryQuadraticModel` from this matrix via
`from_numpy_matrix()`. This method will automatically return a binary model:

In [4]:
to_partition = np.array([1, 4, 5])
interaction_matrix = qubo_partition_matrix(to_partition)
print(interaction_matrix)

[[ -9   4   5]
 [  4 -24  20]
 [  5  20 -25]]


In [5]:
model = dimod.BinaryQuadraticModel.from_numpy_matrix(interaction_matrix)

As you can see, we have used a different constructor than in the example 
above. I personally prefer to define a matrix first, and then pass this 
on to the constructor, instead of fiddling around with dictionaries.

We can now start the sampling. For the sampling, we use the `ExactSolver`,
which solves the QUBO classically on your local CPU. If we use this solver
together with its `sample()` method, we get a result which is comparable to
what D-Wave would give us:

In [6]:
sampleset = dimod.ExactSolver().sample(model)
print(sampleset)

   0  1  2 energy num_oc.
2  1  1  0  -25.0       1
7  0  0  1  -25.0       1
3  0  1  0  -24.0       1
6  1  0  1  -24.0       1
1  1  0  0   -9.0       1
4  0  1  1   -9.0       1
0  0  0  0    0.0       1
5  1  1  1    0.0       1
['BINARY', 8 rows, 8 samples, 3 variables]


Please try this with different sets of numbers $\mathcal{S}$, maybe even
sets where the partition cannot be done neatly into two equal sums.

#### Collecting thoughts
What you might think: if this can be run classically, why take the effort of
running this on a quantum computer? That though might be correct, however,
we _simulate_ the calculation of the QUBO, which takes large amounts of
useless computer power which can be done more optimally in a different way on
a classical computer. As noted in the `dimod` [docs][1], the `ExactSolver`
becomes slow for problems with 18 or more variables.

Furthermore, in Quantum Annealing, we have the power of [Quantum Tunneling][2],
which allows the material which is being annealed to transfer _through_ a
potential hill to a different state, where a classical material would only
be able to reach that state after first having increased the temperature to
an energy higher than said potential hill to pass over it. See the image below
for a clarification.

![quantum tunneling](https://qph.fs.quoracdn.net/main-qimg-674c5917220f56bbaa2d611bb8e1c78f.webp)

[1]: https://docs.ocean.dwavesys.com/projects/dimod/en/latest/reference/sampler_composites/samplers.html?highlight=ExactSolver#dimod.reference.samplers.exact_solver.ExactSolver
[2]: https://en.wikipedia.org/wiki/Quantum_tunnelling

## Appendix
### A1: derivation of $\Delta^2$
Please remember that $x_i = {x_i}^2$, since $x_i \in [0, 1]$.

\begin{align}
    \Delta ^2 &= \left(
                    \mathcal{A} - 2 \sum_i^n x_i s_i
                 \right) ^2 \\
              &= \mathcal{A}^2
                - 4 \mathcal{A} \sum_i^n x_i s_i
                + 4 \sum_{i, j}^n x_i s_i x_j s_j \\
              &= \mathcal{A}^2
                + 4 \left(
                    -\mathcal{A} \sum_i^n {x_i}^2 s_i
                    + \sum_{i=j}^n x_i s_i x_j s_j
                    + \sum_{i\neq j}^n x_i x_j s_i s_j
                \right) \\
              &= \mathcal{A}^2
                + 4 \left(
                    -\mathcal{A} \sum_i^n {x_i}^2 s_i
                    + \sum_i^n {x_i}^2 {s_i}^2
                    + \sum_{i\neq j}^n x_i x_j s_i s_j
                \right) \\
              &= \mathcal{A}^2
                + 4 \left(
                    \sum_i^n {x_i}^2 {s_i}^2 - \mathcal{A} {x_i}^2 s_i
                    + \sum_{i\neq j}^n x_i x_j s_i s_j
                \right) \\
              &= \mathcal{A}^2
                + 4 \left(
                    \sum_i^n {x_i}^2 \left(
                        {s_i}^2 - \mathcal{A} s_i
                    \right)
                    + \sum_{i\neq j}^n x_i x_j s_i s_j
                \right) \\
              &= \mathcal{A}^2
                + 4 \left(
                    \sum_i^n {x_i}^2 s_i \left(
                        s_i - \mathcal{A}
                    \right)
                    + \sum_{i\neq j}^n x_i x_j s_i s_j
                \right) \\
              &= \mathcal{A}^2
                + 4 \left(
                    \sum_i^n {x_i}^2 Q_{ii}
                    + \sum_{i\neq j}^n x_i x_j Q_{ij}
                \right) \\
              &= \mathcal{A}^2
                + 4 Q \cdot (x \otimes x),
\end{align}

where $Q_{ii} = s_i\left(s_i - \mathcal{A}\right)$ and $Q_{ij} = s_i s_j$